In [1]:
pip install transformers datasets torch scikit-learn --break-system-packages


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset

# Load a dataset (replace with your own)
dataset = load_dataset("csv", data_files={"train": "train.csv", "test": "test.csv"})

dataset = dataset.rename_column("is_sarcastic", "labels")


/home/anjali/.local/lib/python3.12/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/anjali/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["headline"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)


/home/anjali/.local/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Convert labels to integers (if necessary)
num_labels = len(set(dataset["train"]["labels"]))


2025-02-21 13:13:46.708160: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-21 13:13:46.878998: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-21 13:13:46.967911: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-21 13:13:46.994091: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-21 13:13:47.130591: I tensorflow/core/platform/cpu_feature_guar

In [5]:
from transformers import TrainingArguments, Trainer, AutoModelForSequenceClassification
import torch.nn as nn

# Load model
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# 🔹 1. Freeze Initial Layers (Only Train Last Few Layers)
for param in model.distilbert.parameters():  
    param.requires_grad = False  # Freeze DistilBERT base layers

# Unfreeze the last transformer layer
for param in model.distilbert.transformer.layer[-1].parameters():
    param.requires_grad = True  # Keep last transformer layer trainable

# 🔹 2. Apply Dropout and L2 Regularization
class CustomDistilBERT(nn.Module):
    def __init__(self, model):
        super(CustomDistilBERT, self).__init__()
        self.model = model
        self.dropout = nn.Dropout(0.3)  # Dropout to prevent overfitting
        self.classifier = nn.Linear(768, 2)  # Adjust for binary classification

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.model.distilbert(input_ids, attention_mask=attention_mask)
        hidden_state = outputs.last_hidden_state[:, 0]  # Take CLS token representation
        dropped = self.dropout(hidden_state)  # Apply dropout
        logits = self.classifier(dropped)  # Classification layer
        return {"logits": logits}

# Replace original model with custom model
model = CustomDistilBERT(model)

# 🔹 3. Set Training Arguments (L2 Regularization + Early Stopping)
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,  # L2 regularization
    load_best_model_at_end=True,  # Load best model using validation loss
    metric_for_best_model="eval_loss",
    logging_dir="./logs",
    logging_steps=10,
    save_steps=1000,
    report_to="none",
)

# 🔹 4. Enable Early Stopping
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],  # Stops if val loss doesn’t improve for 2 epochs
)

# Train the model
trainer.train()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/anjali/.local/lib/python3.12/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/anjali/.local/lib/python3.12/site-packages/torch/cuda/__init__.py:654: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


KeyError: 'validation'

In [21]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=6,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/anjali/.local/lib/python3.12/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [23]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",         # Evaluate at the end of each epoch
    logging_strategy="epoch",            # Log at the end of each epoch
    logging_dir="./logs",                # Directory for storing logs
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from transformers import (DistilBertTokenizerFast, DistilBertForSequenceClassification, 
                          TrainingArguments, Trainer, EarlyStoppingCallback)
import torch.nn as nn
import torch
import numpy as np
import re

# 1. Load your dataset (assuming CSV with columns "headline" and "is_sarcastic")
data = pd.read_csv("train.csv")

# 1a. Convert any dictionary entries in the "headline" column to strings.
def extract_text(x):
    if isinstance(x, dict):
        return x.get("content", str(x))
    return str(x)

data["headline"] = data["headline"].apply(extract_text)

# 1b. Clean the text in the "headline" column
def clean_text(text):
    text = text.lower()
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

data["headline"] = data["headline"].apply(clean_text)

# 2. Split the DataFrame into train and validation (90:10 ratio)
train_df, val_df = train_test_split(data, test_size=0.10, random_state=42, stratify=data["is_sarcastic"])

# 3. Convert Pandas DataFrames into Hugging Face Datasets using Dataset.from_pandas()
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Create a DatasetDict for convenience
datasets = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset
})

# 4. Load the tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

# 5. Tokenize the datasets; note that we use "headline" instead of "text"
def tokenize_function(examples):
    return tokenizer(examples["headline"], padding="max_length", truncation=True)

tokenized_datasets = datasets.map(tokenize_function, batched=True)

# Rename "is_sarcastic" to "labels" so the Trainer can find them.
tokenized_datasets = tokenized_datasets.rename_column("is_sarcastic", "labels")
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# 6. Load the DistilBERT model. The number of labels is determined by the number of unique values in "is_sarcastic".
base_model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", 
    num_labels=len(data["is_sarcastic"].unique())
)

# 7. Freeze all DistilBERT parameters except the last transformer layer.
for param in base_model.distilbert.parameters():
    param.requires_grad = False
for param in base_model.distilbert.transformer.layer[-1].parameters():
    param.requires_grad = True

# 8. Create a custom model wrapper to add dropout.
class CustomDistilBert(nn.Module):
    def __init__(self, base_model, dropout_rate=0.3):
        super(CustomDistilBert, self).__init__()
        self.base_model = base_model
        self.dropout = nn.Dropout(dropout_rate)
        # Use the classifier from base_model.
        self.classifier = base_model.classifier

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.base_model.distilbert(input_ids, attention_mask=attention_mask)
        hidden_state = outputs.last_hidden_state[:, 0]  # Use CLS token representation
        dropped = self.dropout(hidden_state)
        logits = self.classifier(dropped)
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)
        return {"loss": loss, "logits": logits}

# Replace base_model with the custom model
model = CustomDistilBert(base_model)

# 9. Define training arguments with weight decay and early stopping.
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",
)

# 10. Define a simple compute_metrics function (calculating accuracy)
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc}

# 11. Create the Trainer and add EarlyStoppingCallback.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# 12. Train the model.
trainer.train()

# After training, you can evaluate, save the model, or perform inference as needed.


/home/anjali/.local/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map: 100%|██████████| 2290/2290 [00:00<00:00, 6143.55 examples/s]
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/anjali/.local/lib/python3.12/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Trans

{'loss': 0.6927, 'grad_norm': 3.139820098876953, 'learning_rate': 4.992236024844721e-05, 'epoch': 0.01}


  0%|          | 20/6440 [02:32<13:26:20,  7.54s/it]

{'loss': 0.6649, 'grad_norm': 1.815093755722046, 'learning_rate': 4.984472049689442e-05, 'epoch': 0.02}


  0%|          | 30/6440 [03:46<12:36:29,  7.08s/it]

...

 20%|█▉        | 1280/6440 [2:29:47<9:46:40,  6.82s/it] 

{'loss': 0.2669, 'grad_norm': 4.989175319671631, 'learning_rate': 4.006211180124224e-05, 'epoch': 0.99}


                                                       
 20%|██        | 1288/6440 [4:40:53<9:02:14,  6.32s/it]

{'eval_loss': 0.30807387828826904, 'eval_accuracy': 0.8755458515283843, 'eval_runtime': 7814.7459, 'eval_samples_per_second': 0.293, 'eval_steps_per_second': 0.018, 'epoch': 1.0}


RuntimeError: 
            Some tensors share memory, this will lead to duplicate memory on disk and potential differences when loading them again: [{'classifier.weight', 'base_model.classifier.weight'}, {'classifier.bias', 'base_model.classifier.bias'}].
            A potential way to correctly save your model is to use `save_model`.
            More information at https://huggingface.co/docs/safetensors/torch_shared_tensors
            